# Wine dataset

<div>
    <img src="images/wine.jpg" width=400>
</div>

Dalam studi kasus kali ini, anda akan melatih _classifier_ SVM pada dataset **Wine**.    
Dataset dapat diload dengan menggunakan `sklearn.datasets.load_wine()`.
    
Dataset ini berisi analisis kimia dari 178 sampel wine yang diproduksi oleh 3 daerah penghasil yang berbeda.    
Tujuan dataset ini adalah untuk melatih model klasifikasi yang mampu memprediksi daerah penghasil berdasarkan analisis kimia _wine_. Karena  SVM adalah _binary classifier_, anda harus menggunakan _one-versus-all_ untuk mengklasifikasikan ketiga daerah penghasil tersebut.    
> Berapa akurasi yang dapat anda capai?    

Pertama, mari kita load dataset-nya.    
Kita baca deskripsinya, kemudian split dataset menjadi _train set_ dan _test set_.

In [ ]:
from sklearn.datasets import load_wine

wine = load_wine(as_frame=True)

Mari kita cek jumlah instance dari setiap kelas.

In [ ]:
wine.target.value_counts()

target
1    71
0    59
2    48
Name: count, dtype: int64

Dataset tidak terlalu balance tapi masih OK apabila kita menggunakan _accuracy_ sebagai metrik evaluasi.

Mari kita cetak deskripsi dari dataset **Wine**.

In [ ]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            10.6  30.0    19.5   3.3
    Ma

Selanjutnya, mari bagi dataset menjadi train dan test set dengan `train_size` adalah $\frac{3}{4}$ dari dataset dan sisanya menjadi _test set_.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split \
(wine.data, wine.target, train_size=0.75, random_state=42)

Mari kita cetak lima baris pertama dari `X_train`.

In [ ]:
X_train.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
100,12.08,2.08,1.70,17.5,97.0,2.23,2.17,0.26,1.40,3.30,1.27,2.96,710.0
122,12.42,4.43,2.73,26.5,102.0,2.20,2.13,0.43,1.71,2.08,0.92,3.12,365.0
154,12.58,1.29,2.10,20.0,103.0,1.48,0.58,0.53,1.40,7.60,0.58,1.55,640.0
51,13.83,1.65,2.60,17.2,94.0,2.45,2.99,0.22,2.29,5.60,1.24,3.37,1265.0


Mari kita cetak lima baris pertama dari `y_train`.

In [ ]:
y_train.head()

2      0
100    1
122    1
154    2
51     0
Name: target, dtype: int64

Mari kita mulai dengan cara sederhana, yaitu classifier SVM linear atau `LinearSVC`. Secara otomatis, LinearSVC menggunakan strategi _One-vs-All_ (also called _One-vs-the-Rest_, OvR).

In [ ]:
from sklearn.svm import LinearSVC

Mari kita buat kelas `LinearSVC` dengan `random_state` adalah `42`.

In [ ]:
lin_clf = LinearSVC(random_state=42)

Mari kita train `lin_clf` dengan `X_train` dan `y_train`.

In [ ]:
lin_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

Oh no! Anda melihat bahwa `lin_clf` failed to converge.      
Apakah anda dapat menebak mengapa?       
Mungkin solusinya, kita tingkatkan jumlah iterasi trainingnya (`max_iter`)?    
Mari kita gunakan `max_iter` sama dengan `1000000`:

In [ ]:
lin_clf = LinearSVC(random_state=42, max_iter=1000000)

Again, mari kita train `lin_clf` dengan `X_train` dan `y_train`.

In [ ]:
lin_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(max_iter=1000000, random_state=42)

Meskipun dengan satu juta iterasi, proses training masih belum konvergen.    
Pasti ada masalah lain yang menyebabkan hal ini.    

Mari kita evaluasi model ini dengan `cross_val_score`, model `Linear_SVC` ini akan menjadi _baseline_:
  
**Pertanyaan**:   
Berapakah rata-rata dari `cross_val_score`?
* 0.90997150997151

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_clf, X_train, y_train, cv=5)
print(f"Rata-rata cross_val_score: {scores.mean()}")

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Rata-rata cross_val_score: 0.90997150997151


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Akurasi yang dihasilkan pada dataset ini tidaklah bagus. Jadi, apakah anda dapat menebak apa masalahnya?

Benar, anda lupa melakukan normalisasi ($Z$-_normalization_) pada fiturnya! Selalu ingat untuk melakukan normalisasi fitur saat menggunakan SVM:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Mari kita gunakan `make_pipeline` untuk melakukan normalisasi sekaligus membuat `LinearSVC` dengan `random_state=42`.

In [ ]:
lin_clf = make_pipeline(StandardScaler(),\
LinearSVC(random_state=42))

Again, mari kita _train classifier_ `lin_clf`.

In [ ]:
lin_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=42))])

Sekarang proses _training_ sudah _successfully converge without any problems_.    
Mari kita ukur kinerjanya dengan menghitung **rata-rata** hasil `cross_val_score`.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lin_clf,\
X_train, y_train, cv=5)

print("Rata-rata cross-validation score \
setelah normalisasi:", scores.mean())


Rata-rata cross-validation score setelah normalisasi: 0.9774928774928775


Nice! Berapakah _accuracy_ yang diperoleh saat ini?
* 0.9774928774928775

Mari kita lihat apakah SVM yang mempunyai kernel (`SVC`) dapat meningkatkan kinerja dari `LinearSVC`.     
Kita akan menggunakan `SVC` default untuk saat ini:

In [ ]:
from sklearn.svm import SVC

Mari kita buat `make_pipeline` yang terdiri dari:
1. `StandardScaler` dan
2. `SVC` dengan `random_state` adalah 42.

In [ ]:
pipe_svc = make_pipeline(StandardScaler(), \
SVC(random_state=42))

Mari kita hitung hasil fungsi `cross_val_score`.    
Jangan lupa hitung rata-ratanya (`mean`) ya.

In [56]:
svc_scores = cross_val_score \
(pipe_svc, X_train, y_train, cv=5)

print("Rata-rata cross-validation \
score untuk model SVC:", svc_scores.mean())

Rata-rata cross-validation score untuk model SVC: 0.9698005698005698


Hasil yang diperoleh tidak lebih baik, mungkin kita perlu melakukan _hyperparameter tuning_:

In [57]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

Mari kita buat _dictionary_ yang berisi dua hyperparameter, yaitu:
1. `svc__gamma` dengan distribusi `reciprocal` yang memiliki nilai $a=0.001$ dan $b=0.1$
2. `svc__C` dengan distribusi `uniform` yang `loc=1` dan `scale=10`.

In [58]:
param_distrib = {
    "svc__gamma": reciprocal(0.001, 0.1),
    "svc__C": uniform(1, 10)
}

### Pertanyaan Refleksi
1. Jelaskan distribusi _reciprocal_, seperti apa _probability density function_ (pdf), parameter-parameternya apa saja, grafik pdf-nya seperti apa?
2. Jelaskan distribusi _uniform_, seperti apa _probability density function_ (pdf), parameter-parameternya apa saja, grafik pdf-nya seperti apa?

Jawab :

1.Distribusi reciprocal adalah distribusi probabilitas
yang menyatakan bahwa probabilitas sebuah variabel terjadi
berbanding terbalik dengan nilainya. Ini berarti semakin
kecil nilai variabel, semakin besar probabilitasnya,
dan sebaliknya.

Distribusi uniform memiliki fungsi kepadatan probabilitas
(PDF) yang didefinisikan sebagai:

f(x; a, b) = (a / (b^2 * x^2)) * 1_I(a < x < b)

di mana:

- x adalah variabel acak
- a dan b adalah parameter distribusi,
- dengan a > 0 dan b > 0 1_I(a < x < b) adalah
fungsi indikator yang bernilai 1 jika
a < x < b dan 0 sebaliknya

Grafik PDF distribusi reciprocal menunjukkan kurva
yang miring ke kanan dengan puncak di sekitar x = 1/a.
Bentuk kurva dipengaruhi oleh nilai parameter a dan b.

2.Distribusi uniform adalah distribusi probabilitas
di mana setiap nilai dalam rentang tertentu memiliki
probabilitas yang sama untuk terjadi.

f(x; a, b) = 1 / (b - a) * 1_I(a < x < b)

di mana:

- x adalah variabel acak
- a dan b adalah parameter distribusi, dengan a < b
1_I(a < x < b) adalah fungsi indikator yang bernilai
1 jika a < x < b dan 0 sebaliknya

Grafik PDF distribusi uniform menunjukkan garis
horizontal dengan ketinggian 1/(b - a) di antara
batas bawah a dan batas atas b.


Mari kita gunakan `RandomizedSearchCV` dengan `param_distrib` yang sudah dibuat, `n_iter` sebanyak `100`, `cv` sebanyak `5`, dan `random_state=42`.

In [61]:
rnd_search_cv = RandomizedSearchCV \
(pipe_svc, param_distrib, n_iter=100, \
cv=5, random_state=42)

Saatnya kita latih `rnd_search_cv`.

In [62]:
rnd_search_cv.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('svc', SVC(random_state=42))]),
                   n_iter=100,
                   param_distributions={'svc__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c32767ebf70>,
                                        'svc__gamma': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7c3273400220>},
                   random_state=42)

Mari kita tampilkan `classifier` dengan `hyperparameter` terbaik.

In [65]:
best_classifier = rnd_search_cv.best_estimator_
print("Classifier dengan \
hyperparameter terbaik:", best_classifier)


Classifier dengan hyperparameter terbaik: Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc',
                 SVC(C=9.925589984899778, gamma=0.011986281799901188,
                     random_state=42))])


Mari kita tampikan best akurasi dari `rnd_search_cv`.

In [66]:
best_accuracy = rnd_search_cv.best_score_
print("Best accuracy dari \
RandomizedSearchCV:", best_accuracy)

Best accuracy dari RandomizedSearchCV: 0.9925925925925926


Ah, this looks excellent! Let's select this model. Now we can test it on the test set:

In [74]:
test_accuracy = best_classifier.score(X_test, y_test)
print(f"Akurasi pada dataset uji: \
{test_accuracy*100:.2f}% ")

Akurasi pada dataset uji: 97.78% 


### Summary
- This tuned kernelized SVM performs better than the `LinearSVC` model, but we get a lower score on the test set than we measured using cross-validation.
- This is quite common: since we did so much hyperparameter tuning, we ended up slightly overfitting the cross-validation test sets.
- It's tempting to tweak the hyperparameters a bit more until we get a better result on the test set, but this would probably not help, as we would just start overfitting the test set.
- Anyway, this score is not bad at all, so let's stop here.

<center>
    <h1>The End </h1>
</center>